In [1]:
import os
from cliently import client
from cliently.formatter import time_in_ms
import dateutil.tz as dtz
import datetime
from dateutil.relativedelta import relativedelta

# Load your Feedly Access Token and User ID

In [2]:
ACCESS_TOKEN=os.getenv("FEEDLY_ACCESS_TOKEN")
UID = os.getenv("FEEDLY_UID")
BASE_URL = "https://cloud.feedly.com"

# Define a last time to update from

In [3]:
now = datetime.datetime.now(dtz.tzlocal())
last_update = now + relativedelta(weeks=-6)

# Instantiate the client

In [4]:
options = dict(access_token=ACCESS_TOKEN)
a = client.FeedlyClient(**options)

# Choose your feed of interest 

In [5]:
subscriptions = a.get_user_subscriptions()
streamIDs = set(i["categories"][0]["id"] for i in subscriptions)
print(streamIDs)

{'user/6cd10586-58f9-4e23-ae77-8d2e84df40be/category/Journal Blitz'}


In [6]:
jbID=list(streamIDs)[0]

# Update your feed

In [7]:
# mark as read
last_update_formatted = time_in_ms(last_update)
entryIDs = a.get_entry_ids(jbID, newerThan=last_update_formatted, unreadOnly=False)

while "continuation" in entryIDs.keys():
    a.mark_articles_unread(entryIDs["ids"])
    entryIDs = a.get_entry_ids(jbID, continuation=entryIDs["continuation"], 
                               maxcount=1000, newerThan=last_update_formatted) # be sure to keep newerThan!
    
a.mark_articles_unread(entryIDs["ids"])

<Response [200]>

# Response 200 is a success